In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import janitor
import numpy as np
import xgboost as xgb
import smogn
import optuna
import warnings
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold
from sklearn.metrics import  mean_absolute_error,accuracy_score, classification_report, confusion_matrix, roc_auc_score, mean_squared_error, r2_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
#from statsmodels.stats.outliers_influence import variance_inflation_factor
#from statsmodels.tools.tools import add_constant


warnings.filterwarnings('ignore') #Se cargan todas las librerias necesarias para utilizar (Algunas no estan siendo utilizadas porque se utilizaron anteriormente como medida de prevension)

In [ ]:
try:
  filepath = 'datos_sinteticos.csv'
  df= pd.read_csv(filepath)
  df= pd.DataFrame(df)
  df_clean= df.clean_names()
  print(f"El archivo se ha cargado y limpieado correctamente")
except FileNotFoundError:
   print(f"El archivo no ha sido encontrado")  #Cargamos el archivo

In [ ]:
#Informacion general del dataset
print(f'''\nINFORMACIÓN GENERAL DEL DATASET:\n
    - Dimensiones: {df_clean.shape[0]} filas x {df_clean.shape[1]} columnas \n
    - Período de análisis: dataset historico de préstamos bancarios
''')

print(f'''
    \n📊 ESTRUCTURA DEL DATASET: \n{df_clean.info} \n
    \n📈 ESTADÍSTICA DESCRIPTIVA: \n {df_clean.describe()}
    ''')

In [ ]:
#Busco valores faltantes dentro del archivo
print('Valores faltantes:')
missing_values = df_clean.isnull().sum()
missing_percentage = (missing_values / len(df_clean)) * 100
missing_df = pd.DataFrame({
    'Valores Faltantes': missing_values, 
    'Porcentaje': missing_percentage
}).sort_values('Porcentaje',ascending=False)
print(missing_df[missing_df['Valores Faltantes'] > 0])

calificacion_distribution = df_clean['calificacion'].value_counts()
calificacion_percentage = calificacion_distribution.value_counts(normalize=True)  * 100
#Reviso los valores unicos y la distribucion de estos, es decir cuantos hay
print(f'''
    \n📊 · Valores únicos en Calificacion: {df_clean['calificacion'].unique()}\n
    \n📈 · Distribución): \n{calificacion_distribution}''')

In [ ]:

#Otorgo un significado a las variables de la columna que quiero predecir
df_clean['calificacion_label'] = df_clean['calificacion'].map({''
  'A1':'Aprobado',
  'A2':'Aprobado',
  'B1':'Aprobado',
  'C1':'Aprobado con seguimiento',
  'D1': 'No Aprobado',
  'E1': 'No Aprobado'})
label_distribution = df_clean['calificacion_label'].value_counts()
label_percentage = label_distribution.value_counts(normalize=True) * 100

calificacion_distribution = df_clean['calificacion'].value_counts().sort_index()
calificacion_percentage = df_clean['calificacion'].value_counts(normalize=True).sort_index() * 100

print("\n📈 DISTRIBUCIÓN NUMÉRICA:")
for valor in sorted(calificacion_distribution.index):
    print(f"  {valor}: {calificacion_distribution[valor]:,} casos ({calificacion_percentage[valor]:.1f}%)")

In [ ]:
label_distribution = df_clean['calificacion_label'].value_counts()
fig, ax = plt.subplots(2,2, figsize=(15,12))
fig.suptitle('# de calificaciones en archivo', fontsize=12, fontweight='bold')

# Gráfico 1: Cliente vs puntaje final
sns.scatterplot(data=df_clean, x='edad_cliente', y='puntaje', hue='calificacion', alpha=0.6, ax=ax[0,0])
ax[0,0].set_title('Puntaje  vs Edad del Cliente', fontsize=14, fontweight='bold')
ax[0,0].set_xlabel('Edad del Cliente', fontsize=12)
ax[0,0].set_ylabel('Puntaje Final', fontsize=12)
ax[0,0].legend(title='Calificación')


# Gráfico 2: Distribución por edad
ax[0,1].hist(df_clean['edad_cliente'], bins=20, alpha=.7, color='#66b3ff', edgecolor='black')
ax[0,1].set_title('Distribución por Edad', fontsize=14, fontweight='bold')
ax[0,1].set_xlabel('Edad', fontsize=12)
ax[0,1].set_ylabel('Frecuencia', fontsize=12)

# Gráfico 3: Ingresos VS Calificacion
sns.boxplot(data=df_clean, x='calificacion', y='ingresos', palette='Set2', ax=ax[1,0])
ax[1,0].set_yscale('log')
ax[1,0].set_title('Ingresos por Estado de Calificación', fontsize=14, fontweight='bold')
ax[1,0].set_xlabel('Calificación', fontsize=12)
ax[1,0].set_ylabel('Ingresos (escala logarítmica)', fontsize=12)


# Gráfico 4: Relación Deuda-Ingreso VS Default
sns.boxplot(x='mora', y='ingresos', data=df_clean, ax=ax[1, 1], palette=["#de6767", "#1bda91"])
ax[1,1].set_title('Relación Mora e Ingreso', fontsize=14, fontweight='bold')
ax[1,1].set_xlabel('Relación Mora', fontsize=12)
ax[1,1].set_ylabel('Estado de ingreso', fontsize=12)

plt.tight_layout()
plt.show

In [ ]:
#Comienzo con la separacion de los datos que quiero utilizar (x) para predecir la columna objetivo
y= df_clean['calificacion']
x= df_clean[['saldo','ahorro_general','sexo','saldo_por_vencer','saldo_pendiente','provision','dias_mora','edad_cliente','estado_civil','mora','antiguedad_meses','ingresos','descuentos','puntaje']]

In [ ]:
#Prueba sacando valores no relevantes segun maquina
df_test = df_clean[df_clean['puntaje'] <= 500].copy()
Cols_predict = df_test[['saldo','ahorro_general','sexo','saldo_por_vencer','saldo_pendiente','provision','dias_mora','edad_cliente','estado_civil','mora','antiguedad_meses','ingresos','descuentos','puntaje']]
df_balanced = smogn.smoter(data=Cols_predict, y='puntaje')
x_balanced=df_balanced.drop('puntaje', axis=1)
y_balanced=df_balanced['puntaje']

In [ ]:
x= pd.get_dummies(x, drop_first=True) #Asigna valores binarios para los datos con palabras
x_balanced=pd.get_dummies(x_balanced, drop_first=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_balanced)

kmeans = KMeans(n_clusters=9, random_state=42)
x_balanced['cluster'] = kmeans.fit_predict(X_scaled)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) #Dividimos los datos 80% entrenamiento 20% prueba
x_train2, x_test2, y_train2, y_test2 = train_test_split(x_balanced, y_balanced, test_size=0.1, random_state=42)

In [ ]:
#Genero un pipeline (defino una cadena de eventos para el momento que lo llame) con el que pueda entrenar, modelar, equilibrar, ejecutar y medir su score para saber que tan acertiva es.
def pipeline(trial):
    n_estimators = trial.suggest_int("n_estimators",50,70)
    max_depth= trial.suggest_int("max_depth",25,32)
    min_samples_split= trial.suggest_int("min_samples_split",15,25)
    min_samples_leaf= trial.suggest_int("min_samples_leaf",4,8)
    
    pipeline = imbpipeline([
            ('Sm',SMOTE(random_state=42)),
            ('RFC',RandomForestClassifier( 
                n_estimators= n_estimators,         
                max_depth=max_depth,            
                min_samples_split=min_samples_split,     
                min_samples_leaf=min_samples_leaf,       
                max_features=0.2,      
                random_state=42,
                n_jobs=-1,
                class_weight='balanced'
            ))
        ])
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, x_train, y_train, cv=skf, scoring='accuracy')
    return scores.mean()

In [ ]:
#def pipeline_lineal(trial):
#    n_estimators = trial.suggest_int("n_estimators",80,160)
#    max_depth= trial.suggest_int("max_depth",60,80)
#    min_samples_split= trial.suggest_int("min_samples_split",25,40)
#    min_samples_leaf= trial.suggest_int("min_samples_leaf",10,20)
#    
#    pipeline2 = imbpipeline([
#                ('RFC',RandomForestRegressor(
#                n_estimators = n_estimators,
#                max_depth=max_depth,            
#                min_samples_split=min_samples_split,     
#                min_samples_leaf=min_samples_leaf,       
#                max_features=0.2,      
#                random_state=42,
#                n_jobs=-1,
#            ))
#        ])
#    kf = KFold(n_splits=5, shuffle=True, random_state=42)
#    scores2 = cross_val_score(pipeline2, x_train2, y_train2, cv=kf, scoring='r2')
#    return scores2.mean()

def objetivo(trial):
   n_estimators= trial.suggest_int('n_estimators',100,400)
   learning_rate= trial.suggest_float('learning_rate',0.01,0.4, log=True)
   max_depth=trial.suggest_int('max_depth',3,20)
   subsample= trial.suggest_float('subsample',0.5,1.0)
   colsample_bytree= trial.suggest_float('colsample_bytree',0.3,1.0)
   gamma=trial.suggest_float('gamma',0,5.0)
   reg_alpha=trial.suggest_float('reg_alpha',0.0,10.0)
   reg_lambda= trial.suggest_float('reg_lambda',0.0,10.0)
        
   modelo= xgb.XGBRegressor(
      n_estimators=n_estimators,
      max_depth=max_depth,
      learning_rate=learning_rate,
      subsample=subsample,
      colsample_bytree=colsample_bytree,
      gamma=gamma,
      reg_alpha=reg_alpha,
      reg_lambda=reg_lambda,
      random_state=42,
      n_jobs=-1
   )
   kf = KFold(n_splits=5, shuffle=True, random_state=42)

   score = cross_val_score(modelo,x_train2,y_train2, cv=kf, scoring='r2')
   
   return score.mean()

In [ ]:
#Utilizo optuna para que me ayude a sacar la mejor version posible del pipeline con 150 intentos.
study = optuna.create_study(direction="maximize")
study.optimize(pipeline, n_trials=5)

study2 = optuna.create_study(direction="maximize")
study2.optimize(objetivo, n_trials=300) 

#study3 = optuna.create_study(direction="maximize")
#study3.optimize(pipeline_lineal, n_trials=60)



In [ ]:
#Encapsulo los mejores parametros y los utilizo para el modelo de machine learning
best_parameters=study.best_params
best_parameters2=study2.best_params
#best_parameters3=study3.best_params

pipeline = imbpipeline([
    ('Sm',SMOTE(random_state=42)),
    ('RFC',RandomForestClassifier( 
        n_estimators= best_parameters['n_estimators'],         
        max_depth=best_parameters['max_depth'],            
        min_samples_split=best_parameters['min_samples_split'],     
        min_samples_leaf=best_parameters['min_samples_leaf'],       
        max_features=0.2,      
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
      ))
])

modelo = xgb.XGBRegressor( 
        n_estimators= best_parameters2['n_estimators'],         
        max_depth=best_parameters2['max_depth'],     
        learning_rate=best_parameters2['learning_rate'],     
        subsample=best_parameters2['subsample'],
        colsample_bytree=best_parameters2['colsample_bytree'],
        gamma=best_parameters2['gamma'],
        reg_alpha=best_parameters2['reg_alpha'],
        reg_lambda=best_parameters2['reg_lambda'],      
        random_state=42,
        n_jobs=-1,
      )

def Linealpipeline_opt (trial):
  k = trial.suggest_int('k', 3, x_train2.shape[1])

  pipeline_xgb = imbpipeline([('scaler',StandardScaler()),
    ('select',SelectKBest(f_regression, k=k)),
    ('model',modelo)])
  
  pipeline_xgb.fit(x_train2,y_train2)

  y_pred2 = pipeline_xgb.predict(x_test2)
  r2 = r2_score(y_test2,y_pred2)
  return r2

  
#pipeline2 = imbpipeline([
#                ('RFC',RandomForestRegressor(
#                n_estimators = best_parameters3['n_estimators'],
#                max_depth=best_parameters3['max_depth'],        
#                min_samples_split=best_parameters3['min_samples_split'],  
#                min_samples_leaf=best_parameters3['min_samples_leaf'],       
#                max_features=0.2,
#                random_state=42,
#                n_jobs=-1,
#            ))
#        ])

#score_lineal= cross_val_score(pipeline2,x_train2,y_train2,scoring='r2', cv=10)
#print(f"Score de r2 sobre los datos {score_lineal.mean()}")

In [ ]:
study3 = optuna.create_study(direction="maximize")
study3.optimize(Linealpipeline_opt, n_trials=15)

In [ ]:
#Lo entreno y comienzo a realizar las pruebas de que tan acertivo es
pipeline.fit(x_train, y_train)
modelo.fit(x_train2, y_train2)

#Predicciones con los modelos
y_predRFC = pipeline.predict(x_test)
y_probaRFC = pipeline.predict_proba(x_test)[:, :]
y_predRFC2 = modelo.predict(x_test2)


In [ ]:
importances = modelo.feature_importances_
plt.barh(x_train2.columns, importances)
plt.title("Importancia de características")

plt.show()

In [ ]:
#vuelvo a medir el modelo con los datos de entrenamiento, pero con el mejor pipeline para saber que tanto puede equivocarse
scores= cross_val_score( pipeline, x_train,y_train, scoring='accuracy')
scores2= cross_val_score( modelo, x_train2,y_train2, scoring='r2')
#Matriz de Confusion
#con esto mido que tanto se ha podido equivocar el modelo
disp = ConfusionMatrixDisplay(confusion_matrix(y_test,y_predRFC))
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusión")


In [ ]:
print("Accuracy (RFC):", accuracy_score(y_test, y_predRFC)) #Compara las precisiones

print("Reporte de clasificación (RFC):\n", classification_report(y_test, y_predRFC)) #Puntuacion de datos por clase 

print("Matriz de confusión (RFC):\n", confusion_matrix(y_test, y_predRFC)) #Comparacion de aciertos por modelos

print("ROC AUC (RFC):", roc_auc_score(y_test, y_probaRFC, multi_class="ovr")) # Hace una comparacion entre los verdaderos positivos y los falsos positivos (Roc: Receiver operating Characteristics; Mide el area bajo la curva, a mayor cercania de 1 mejor clasificacion (Auc: Area under the Curve))

print("Los mejores parametros son: ", best_parameters)
print("Accuracy en cada fold:", scores)
print("Accuracy promedio:", np.mean(scores))
print("Desviación estándar:", np.std(scores))


print("Maquina de regresion lineal")

mse = mean_squared_error(y_test2,y_predRFC2)
mser = np.sqrt(mse)

print("R2 Score:", r2_score(y_test2, y_predRFC2))                # Qué tan bien el modelo explica la varianza
print("MAE:", mean_absolute_error(y_test2, y_predRFC2))          # Error promedio absoluto
print("RMSE:", mser)  # Raíz del error cuadrático medio


print("Accuracy en cada fold:", scores2)
print("Accuracy promedio:", np.mean(scores2))
print("Desviación estándar:", np.std(scores2))

In [ ]:
#Mido cuantos datos fueron a cada uno, es decir de todas las filas, cuales pertenecen a cada dato, y la relevancia de cada columna para el resultado.
unique, counts = np.unique(y_train2, return_counts=True)
print(dict(zip(unique, counts)))
print(np.unique(y_train2, return_counts=True))

importancia = modelo.feature_importances_
columnas = x_train2.columns

df_list = pd.DataFrame({'Variables' : columnas, 'Importancia': importancia})
df_list = df_list.sort_values(by='Importancia', ascending=False)

print(df_list)